# BERT MODEL 

In [1]:
!nvidia-smi

Mon Jun 13 18:34:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   34C    P0    39W / 300W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# CUDA error
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric

In [5]:
code_folder = '/home/ubuntu/HwangJaewon/DL_class/code/'
problem_folders = os.listdir(code_folder)


In [6]:
MODEL = "klue/bert-base"
INPUT = "/home/ubuntu/HwangJaewon/DL_class/sample_train.csv"
MAX_LEN = 256
dataset = load_dataset("csv", data_files=INPUT)['train']
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Using custom data configuration default-007c09a776df80ce
Reusing dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-007c09a776df80ce/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
def example_fn(examples):
    outputs = tokenizer(examples['code1'], examples['code2'], padding=True, max_length=MAX_LEN,truncation=True)
    if 'similar' in examples:
        outputs["labels"] = examples["similar"]
    return outputs

dataset = dataset.map(example_fn, remove_columns=['code1', 'code2', 'similar'])
dataset = dataset.train_test_split(0.1)

_collator = DataCollatorWithPadding(tokenizer=tokenizer)
_metric = load_metric("glue", "sst2")

def metric_fn(p):
    preds, labels = p
    output =  _metric.compute(references=labels, predictions=np.argmax(preds, axis=-1))
    return output
    
model = BertForSequenceClassification.from_pretrained(MODEL) # RobertaForSequenceClassification 는 BertForSequenceClassification 와 달리 pooler가 없는게 기본이기 때문에 문장 유사도에 사용 가능.

args = TrainingArguments(
    'runs/',
    per_device_train_batch_size=32,
    num_train_epochs=3,
    do_train=True,
    do_eval=True,
    save_strategy="epoch",
    logging_strategy="epoch",
    evaluation_strategy="epoch",
)
trainer = Trainer(
        model=model,
        args=args,
        data_collator=_collator,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer,
        compute_metrics=metric_fn)

trainer.train()

Parameter 'function'=<function example_fn at 0x7f24fa467790> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/17970 [00:00<?, ?ex/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

Epoch,Training Loss,Validation Loss,Accuracy
1,0.301000,0.161925,0.938787
2,0.110000,0.144202,0.960490
3,0.035300,0.095587,0.976628


***** Running Evaluation *****
  Num examples = 1797
  Batch size = 8
Saving model checkpoint to runs/checkpoint-506
Configuration saved in runs/checkpoint-506/config.json
Model weights saved in runs/checkpoint-506/pytorch_model.bin
tokenizer config file saved in runs/checkpoint-506/tokenizer_config.json
Special tokens file saved in runs/checkpoint-506/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1797
  Batch size = 8
Saving model checkpoint to runs/checkpoint-1012
Configuration saved in runs/checkpoint-1012/config.json
Model weights saved in runs/checkpoint-1012/pytorch_model.bin
tokenizer config file saved in runs/checkpoint-1012/tokenizer_config.json
Special tokens file saved in runs/checkpoint-1012/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1797
  Batch size = 8
Saving model checkpoint to runs/checkpoint-1518
Configuration saved in runs/checkpoint-1518/config.json
Model weights saved in runs/checkpoint-1518/pytorch_model.bin
t

TrainOutput(global_step=1518, training_loss=0.14875806398856625, metrics={'train_runtime': 815.5222, 'train_samples_per_second': 59.494, 'train_steps_per_second': 1.861, 'total_flos': 6382942647505920.0, 'train_loss': 0.14875806398856625, 'epoch': 3.0})

In [8]:
import pandas as pd

TEST = "/home/ubuntu/HwangJaewon/DL_class/test.csv"
SUB = "/home/ubuntu/HwangJaewon/DL_class/sample_submission.csv"

test_dataset = load_dataset("csv", data_files=TEST)['train']
test_dataset = test_dataset.map(example_fn, remove_columns=['code1', 'code2'])

predictions = trainer.predict(test_dataset)


Using custom data configuration default-274b9386f5046d48
Reusing dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-274b9386f5046d48/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/csv/default-274b9386f5046d48/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-1c80317fa3b1799d.arrow
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: pair_id. If pair_id are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 179700
  Batch size = 8


FileNotFoundError: [Errno 2] No such file or directory: 'submissions/submission.csv'

In [9]:
df = pd.read_csv(SUB)
df['similar'] = np.argmax(predictions.predictions, axis=-1)
df.to_csv('submission.csv', index=False)



In [10]:
df.head()

,pair_id,similar
0,1,0
1,2,1
2,3,1
3,4,0
4,5,0


In [ ]:
df = pd.read_csv(SUB)
df['similar'] = np.argmax(predictions.predictions, axis=-1)
df.to_csv('submissions/submission.csv', index=False)